# Проработка вариантов решения

Задание:
- проанализировать имеющиеся решения, обозначить плюсы и минусы каждого из потенциальных решений;
- выбрать подходящее решение для нашей задачи и обосновать его;
- соотнести ML-метрики с прокси-метриками и обосновать выбор;
- сформировать обучающую выборку;
- спроектировать схему валидации с учетом специфики задачи.

# Анализ имеющихся решений

Мы пойдем по следующему пути: проанализируем модели и алгоритмы в работах, которые пытались решить задачи категориальной классификации/продуктовой кластеризации/детекции товаров для взрослых в электронной коммерции. Статей в данной тематике много, поэтому ограничимся небольшой выборкой (всё-таки не научную работу проводим).

## Ye Bi et al. (2017)
Авторы участвовали в соревновании `SIGIR-2020`, где необходимо было провести классификацию товара (отнести к соответствующему номеру категории) используя заголовок, описание и изображение. Классы сильно несбалансированы.

В работе в качестве ***кодировщика текста*** использовалась предобученная на большом корпусе французских текстов модель `CamemBERT` (необходимо было работать с текстами на французском). На вход модели подавался сконкатенированный текст заголовка и описания, разделенные при помощи тега `[SEP]`. Предобработка текста была достаточно простой: были удалены лишние пробелы и HTML-тэги. На выходе получался вектор размера 768 (base модель) или 1024 (large модель).

В качестве ***кодировщика изображений*** использовалась модель `ResNet152`, предобученная на датасете `ImageNet`. Изображения вначале очищались от шума (удалялись пустые или одноцветные изображения, сломанные изображения) с помощью `cleanlab`. Дополнительно проводилась аугментация: добавлялись изображения с клипованием, кадрированием, вращением и  горизонтальным смещением. Изображения на вход подавались с разрешением (428 x 428).

Слияние результатов работы кодировщиков проводилось с использованием двух схем: `feature-level fusion` и `decision-level fusion`. В первом случае вектора, полученные от каждого кодировщика, преобразуются в один вектор (конкатенацией, суммированием и т.п.), к полученному вектору затем применяется классификатор на основе полносвязной нейронной сети. Авторам не удалось построить удачную модель по данной схеме, все результаты были хуже, чем для текстового классификатора в отдельности. Вторая схема предполагает, что для каждого кодировщика имеется свой отдельный классификатор. На полученных данных затем сроится итоговая модель классификации. В данной работе `decision-level fusion` модель показала наилучшие результаты по метрике `Macro-F1`. Команда выиграла соревнование, заняв первое место на финальном лидерборде с результатом 0.914.

![[DLS]](https://i.postimg.cc/4dDkWWq6/Bi-model.png)


## Wirojwatanakul et al. (2019)
Решалась задача категоризации товаров из Amazon. Выборка содержала 119073 товара, 90000 из которых использовались для обучения. Каждый товар в выборке мог быть отмечен метками нескольких категорий. Авторы провели следующую предобработку: удалили категории, в которых находилось менее 400 товаров. После этого, каждый товар в среднем имел 3 категории, максимальное и минимальное количество товаров на категорию были 37102 и 558, соответственно. Классы были сильно несбалансированы. 

Для оценки качества модели авторы использовали `micro-averaged F1` метрику, которая применима для классификации по нескольким меткам и для несбалансированных классов. В данной работе кодирование заголовка и описания проводилось по-отдельности:
- ***Description classifier***: из описания удялялись стоп-слова, лишние пробелы, числа, пунктуация, и слова длиннее 30 символов. Описания усекались (дозаполнялись) до 300 слов. Классификатор имел следующую структуру: эмбеддинги получались при помощи GloVe (покрывал 61% словаря), сверточный слой (размер ядра 5, 200 фильтров), глобальный max pooling, полносвязный слой (170 нейронов, активация `ReLU`), полносвязный слой (122 нейрона, активация `sigmoid`). Результат на тесте - 77%. 
- ***Title classifier***: стоп-слова *не удалялись*, заголовок приводился к размеру в 57 слов. Нейронная сеть была аналогичной классификатору по описанию. Результат на тесте - 82.7%.
- ***Image classifier***: предобученная на `ImageNet` модель `ResNet50`, адаптированная для классификации по 122 классам. При обучении первые слои замораживались, их веса не обновлялись (у авторов получилось, что оптимальным является 40 замороженных слоев). Результат на тесте - 61%.

Среди унимодальных моделей лучшей оказалась `CNN`, обученная на заголовках.

Перечисленные классификаторы были объединены по схеме `late fusion`. Авторы провели сравнение bi-modal fusion (image-description, image-title, description-title) и tri-modal fusion моделей. Результат по F1: 82%, 85%, 87% и 88.2%, соответственно. В качестве итогового классификатора использовалась трехслойная полносвязная нейронная сеть (200, 150 и 122 нейронов на слой; функции активации: `sigmoid`, `tanh`, `sigmoid`, соответственно).

<img src="https://i.postimg.cc/J4WCJdbm/Wiroj-model.png" width="550">

В заключении авторы в качестве замены `CNN` для классификации текста предлагают обратиться к `RNN` и трансформерам.


## Ekansh Verma (2020)
Авторы участвовали в том же соревновании, что и Ye Bi et al. (2018): *2020 SIGIR Workshop On eCommerce (ECOM20)*. Также как и в рассмотренной ранее работе, текст описания и текст заголова были объединены. Картинки приводились к размеру (224 x 224), нормализовались по каналу. Также проводилась аугментация схожим образом. Метрика качества - `Macro-F1`.

Вначале в качестве baseline авторы обучили отдельно классификаторы только на текстовых данных и только на изображениях:
- ***Image classifier***: в качестве кодировщика использовалась модель `SE-ResNeXt` c 50 слоями, предобученная на датасете `ImageNet`. Затем применялись адаптивный average pooling слой и линейный слой с числом нейронов, равным числу классов. Результат на валидационной части - 61.44%.
- ***Text classifier***: `FlauBERT` или `CamemBERT` + классификатор, схожий с классификатором для изображений. Результат на валидационной части - 89.37% и 89.21% соответственно. 

Схема построения fusion модели представлена на картинке. Авторы использовали и `feature-level fusion` и `descion-level fusion` схемы: 
- `early fusion`: Выходы кодировщика изображений прогонялись через 1D сверточный слой, чтобы размер вектора был равен размерам векторов от `BERT` моделей. Вектора от каждой модели  конкатенировались в один (также был рассмотрен вариант сложения всех векторов). Схема представлена на картинке. Результат - около 90.5%.
- `late fusion`: выходы классификаторов от трех моделей подавались на вход `LightGBM`, который проводил итоговую классификацию. Результат - 91.96%.

Лучшей моделью оказалась `late fusion` с классификатором `LightGBM`. На итоговом лидерборде команда получила 90.53% и заняла третье место.

<img src="https://i.postimg.cc/8kvV7Vft/Verma-model.png" width="650">



## Shin et al. (2022)
Авторы разработали фреймворк обучения, который согласовывает немаркированные языковые и визуальные представления продукта. Качество оценивалось в рамках задач категориальной классификации, извлечения атрибутов, сопоставления продуктов и распознавания продуктов для взрослых. Данные для работы брались из онлайн магазина NAVER. 

*Предобработка данных*: были удалены пары изображение-текст без картинок, с пустыми картинками, слишком маленькими изображениями и сломанными изображениями, с заголовками не относящимися к картинке, и с заголовками, в которых менее двух токенов. Дубликаты по заголовкам и по картинкам также удалялись. Исключались из рассмотрения неправильно промаркированные товары "для взрослых", где, например, часть изображения была заблюрена а is_adult=False. Итоговый датасет содержал 270М продуктов, 41К использовались в качестве тестовых данных.

![[Shin]](https://i.postimg.cc/KjtXwnhd/Shin-model.png)

- ***Image encoder***: ViT-B/32 трансформер. По сравнению с CNN требует меньше времени на обучение и меньше GPU в мультимодальных предобученных задачах с двумя энкодерами.
- ***Text encoder***: многоязычная BERT модель.

Кодировщики обучаются совместно с использованием `symmetric cross-entropy loss` за счет максимизации косинусной схожести эмбеддингов пар текст-изображений, относящихся к одному товару, и минимизации схожести для пар, относящихся разным товарам. В каждом батче при обучении не должно быть дубликатов.

Оценка проводилась с использованием трех подходов:
1. `zero-shot-transfer` - для каждого названия класса получают эмбеддинги, которые затем сравниваются с усредненными эмбеддингами от пар текст-картинка. В итоге для каждой интересующей можно найти наиболее вероятный класс, к которому она принадлежит.
2. `linear probe` - обученная модель замораживается, обучается только линейный классификатор, построенный на получаемых эмбеддингах от пар текст-картинка.
3. `fine-tuning` - помимо линейного классификатора, энкодеры дополнительно дообучаются.

**Задача: кластеризация**<br>
размерность полученных мультимодальных эмбеддингов понижалась до 128 с помощью `PCA`, затем кластеризация проводилась с использованием алгоритма `k-means`. Оценка кластеров проводилась с использованием метрик *точность кластеризации* `ACC`, *нормализованной взаимной информации* `NMI` и *скорректированного индекса Рэнда* `ARI`.

**Задача: категориальная классификация**<br>
оценка категориальной классификации проводилась по трем датасетам: со всеми категориями, модные товары, товары для детей. Количество продуктов в каждой категории нормализовалось. Метрика - `accuracy`.

**Задача: распознование товаров для взрослых**<br>
тренировочные данные: 10К товаров для взрослых и 50К обычных товаров. Тестовые: 1К к 5К. Метрика - `F1-score`.

Согласно приведенным в статье результатам, предложенная авторами модель превосходит стандартные методы CLIP, KELIP.

# Выбор модели и её обоснование 

Основные требования к модели:
- Кодировщик текстового описания:
    - русскоязычность/мультиязычность;
    - размер входного текста: не менее 200 токенов (среднее число слов в описании - 200);
    - предобучен на большом корпусе русских текстов;
- Кодировщик изображения:
    - разрешение картинки при подаче на вход кодировщику должно быть относительно большим;
    - предобучен на большом количестве изображений;
- Для всей мультимодальной модели:
    - должна обучаться на немаркированных данных, только с использованием пары текст-изображение;
    - должна обучаться на доступных ресурсах (kaggle, colab) за адекватное время (~ час);
    - должна быть легкодоступной (на PyTorch или Huggingface).

Для обучения на немаркированных данных будем использовать `contrastive learning`, который используется в `CLIP`. Имеется модель `ruCLIP`, которая работает с русским текстом, однако контекстное окно у нее совсем небольшое - 77 токенов. Необходимо подобрать каждый энкодер самостоятельно.

Попытки сравнить энкодеры по качеству и производительности проводились неоднократно. Для энкодеров, которые могут работать с русским текстом, составлен неплохой рейтинг [здесь](https://github.com/avidale/encodechka?tab=readme-ov-file), который складывался из мреднего качетсва работы, скорости работы на CPU и GPU, а также занимаемой памяти. Для решения нашей задачи могут подойти следующие варианты:
- `rubert-tiny2` (2048)
- `LaBSE-en-ru` (256)
- `MUSE-3`

В качетве кодировщика изображений можно использовать:
- `EfficientNet-B3` (300 x 300)
- `ViT-B/32` (224 x 224)
- `ResNet-152` (224 x 224)

# Метрики

## Задача кластеризации

Ранжирование товаров по степени схожести.

- *adjusted mutual information* `AMI`: скорректированная взаимная информация. Интуитивно, взаимная информация `MI` измеряет долю информации, общей для обоих разбиений: насколько информация об одном из них уменьшает неопределенность относительно другого. `AMI` позволяет избавиться от роста индекса `MI` с увеличением числа классов. Принимает значения в области $[0, 1]$. Значения, близкие к нулю, говорят о независимости разбиений, а близкие к единице – об их схожести. Для расчета необходимы истинные метки.
- *гомогенность*, *полнота* и *V-мера*: гомогенность измеряет, насколько каждый кластер состоит из объектов одного класса, а полнота — насколько объекты одного класса относятся к одному кластеру. V-мера - гармоническое среднее гомогенности и полноты.
- *Силуэт*: подходит для оценки качества кластеризации как таковой, без какой-либо информации об истинных классах.

## Задача категориальной классификации

Отнесение товара к категории, подкатегории. Для одежды отнесение к сезонности товара. Для какого пола товар предназначен и какие имеются возрастные ограничения.

- `macro-F1`: для каждого класса по отдельности рассчитывается `F1`, затем берется среднее арифметическое всех полученных значений;
- `weighted-averaged-F1`: также для каждого класса рассчитывается `F1`, затем берется среднее взвешенное. Веса берутся на основе истинного распределения элементов по классам;
- `accuracy` или `micro-F1`: для каждого класса рассчитываются `TP`, `FP`, `FN`, затем данные значения складываются и итоговое значение `micro-F1` рассчитывается по формуле: $TP/(TP+0.5(FP+FN))$. То же самое значение получится и для `accuracy`. Подходит только для сбалансированных выборок.

## Задача бинарной классификации

Является ли товар товаром для взрослых. Является ли товар хрупким.

- Стандартный `F1-score`

# Формирование обучающей выборки

In [2]:
import re
import string
import pandas as pd
import numpy as np
import zipfile
from PIL import Image

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import swifter
from tqdm.notebook import tqdm

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

tqdm.pandas()

# сделаем возможным параллельную обработку строковых колонок в pd.DataFrame
swifter.set_defaults(
    allow_dask_on_strings=True,
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sergei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# предобработка текста в заголовке и описании
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self, max_word_len=15, title_size=60, description_size=300):
        self.title_size = title_size
        self.description_size = description_size
        self.max_word_len = max_word_len
        
        units = ['мг', 'г', 'гр', 'кг', 'мл', 'л', 'мм', 'см', 'м', 'км', 'шт', 'штук']
        stop_words = stopwords.words('russian')
        
        # часть из стоп-слов оставим
        stop_words.remove('не')
        stop_words.remove('для')
        
        self.title_exclude_list = list(string.punctuation) + units
        self.description_exclude_list = self.title_exclude_list + stop_words
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None, get_characteristics=True):
        X = X.copy()
        
        X['title'] = X['title'].swifter.progress_bar(desc='Title process').apply(lambda x: self._process_text(x, 'title'))
        X['description'] = X['description'].swifter.progress_bar(desc='Description process').apply(lambda x: self._process_text(x, 'description'))
        
        if get_characteristics:
            X['characteristics'] = X['characteristics'].progress_apply(lambda x: self._process_characteristics(x))
            
        return X
    
    def _process_text(self, text: str, text_type: str) -> str:
        
        if text_type == 'title':
            exclude_list = self.title_exclude_list
            size = self.title_size
            
        elif text_type == 'description':
            exclude_list = self.description_exclude_list
            size = self.description_size
            
        else:
            raise ValueError('text_type must be \'title\' or \'description\'')
        
        # удаляем сочетания `<число> x <число>` и `<число><единица измерения>`
        text = re.sub(r'\d+[хХxX]\d+|\d+[кмглршт]+|\d+ [xXхХ] \d+', '', text)
            
        return ' '.join([word for word in word_tokenize(text.lower()) if word not in exclude_list and 
                                                                      not word.isdigit() and
                                                                      len(word) <= self.max_word_len][:size])
    
    def _process_characteristics(self, text: str) -> list[dict]:
        text = text.decode('utf-8')

        # первый паттерн
        if '\'b\\\'' in text:
            return eval(re.sub(r'\\+', r'\\', text)[5:-3].replace('true', 'True').replace('false', 'False'))

        # второй паттерн
        text = text.replace("}\n", "},")
        text = re.sub(r'\n\s+', ' ', text)
        text = re.sub(r'array', 'np.array', text)

        return eval(text)

In [4]:
# извлечение характеристик
class CharExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, char_names: list = None, col_names: list = None):
        if char_names is None and col_names is None:
            self.char_names = ['Пол', 'Возрастные ограничения', 'Сезон', 'Хрупкость']
            self.col_names = ['sex', 'age_restrictions', 'season', 'fragility']
            
        elif char_names is None or col_names is None:
            raise ValueError('Both char_names and col_names must be provided')
            
        else:
            self.char_names = char_names
            self.col_names = col_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        X[self.col_names] = pd.DataFrame(X['characteristics'].swifter.apply(lambda x: self._get_characteristic(x)).tolist(), 
                                         columns=self.char_names, index=X.index)
        
        return X
    
    def _get_characteristic(self, char_list: list[dict]) -> float|list[float]:
        res = [np.nan] * len(self.char_names)

        for char in char_list:
            
            if char['charcName'] in self.char_names:

                if 'value' in char and char['value'] is not None:
                    if isinstance(char['value'], str):
                        char['value'] = float(char['value'].replace(',', '.'))

                    res[self.char_names.index(char['charcName'])] = char['value']

                if 'charcValues' in char and char['charcValues'] is not None:
                    if len(char['charcValues']) == 1:
                        res[self.char_names.index(char['charcName'])] = char['charcValues'][0]
                    else:
                        res[self.char_names.index(char['charcName'])] = list(char['charcValues'])
                    
        return res

In [8]:
image_df = pd.read_parquet('image_df.parquet')

# фильтр для пустых изображений
empty_nms = image_df[image_df['size'] < 25 * 1024].nm

In [5]:
path_to_files = 'D:/HorizontalML/'
path_to_zip = path_to_files + 'wb_school_horizcv_images.zip'

In [6]:
def clean_dataset(df, cols_to_drop=['brand', 'price']):
    df = df.copy()
    
    df.loc[df.title == '', 'title'] = np.nan
    df.loc[df.description == '', 'description'] = np.nan
    
    # удаляем товары, где нет названия и описания
    df = df[df[['title', 'description']].notna().all(axis=1)]
    
    df = df.copy()
    
    # заполняем пропуски в описаниях заголовком, где это возможно
    df['description'] = df.description.fillna(df.title)
    
    # заполням пропуски в заголовках первыми 10 словами из описания
    df['title'] = df['title'].fillna(df['description'].str.split().str[:10].str.join(' '))
    
    # исключаем товары с пустыми картинками
    df = df[~df.nm.isin(empty_nms)]
    
    # удалим товары, заголовок которых состоит менее чем из 2х символов
    df = df[df['title'].str.len() > 2]
    
    # удаляем колонки, которые не будут использованы
    for col in df.columns:
        if col in cols_to_drop:
            df = df.drop(col, axis=1)
    
    return df

In [7]:
train = clean_dataset(pd.read_parquet(path_to_files + 'wb_school_train.parquet'))
test = clean_dataset(pd.read_parquet(path_to_files + 'wb_school_test.parquet'))

In [8]:
preprocessor = TextPreprocessor()
char_extractor = CharExtractor()

train = char_extractor.transform(preprocessor.transform(train))
test = preprocessor.transform(test, get_characteristics=False)

Title process:   0%|          | 0/24 [00:00<?, ?it/s]

Description process:   0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/98660 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/98660 [00:00<?, ?it/s]

Title process:   0%|          | 0/24 [00:00<?, ?it/s]

Description process:   0%|          | 0/24 [00:00<?, ?it/s]

In [9]:
train.loc[train.description == '', 'description'] = train.loc[train.description == '', 'title']
train.loc[train.title == '', 'title'] = train.loc[train.title == '', 'description'].apply(lambda x: ' '.join(x.split()[:7]))

test.loc[test.description == '', 'description'] = test.loc[test.description == '', 'title']
test.loc[test.title == '', 'title'] = test.loc[test.title == '', 'description'].apply(lambda x: ' '.join(x.split()[:7]))

## Возрастные ограничения

In [63]:
def get_age_restrictions(ar):
    if ar in ['0+', '0 +']:
        return np.nan
    
    ar = ar.lower()
    
    if re.findall(r'без огран|нет огран|для всех|любо[йг]', ar):
        return 'для всех возрастов'
    
    if re.findall(r'мес|годик|рожден|малыш|реб[её]н', ar):
        return 'для малышей'
    
    nums = sorted(re.findall(r'\d+', ar))
    
    if nums:
        num = int(nums[0])
        if num >= 18:
            return 'для взрослых'
        elif 12 <= num < 18:
            return 'для подростков'
        elif 3 <= num < 12:
            return 'для детей'
        else:
            return 'для малышей'
        
    if re.findall(r'дет[ямскей]{2}|школ|девоч|мальчик', ar):
        return 'для детей'
    
    if re.findall(r'мужч|женщ|взросл', ar):
        return 'для взрослых'
    
    if re.findall(r'подрост', ar):
        return 'для подростков'
        
    return 'для всех возрастов'

In [66]:
train['age_restrictions'] = train.age_restrictions.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
train.age_restrictions = train.age_restrictions.dropna().apply(get_age_restrictions)

In [67]:
train.age_restrictions.value_counts()

для детей             1910
для подростков         882
для малышей            632
для взрослых           316
для всех возрастов     165
Name: age_restrictions, dtype: int64

## Пол

Классификация по половым признакам:
- Мужская
    - мужская
    - для мальчиков
- Женская
    - женская
    - для девочек
- Унисекс
- Детская

Исходя из этой классификации, оставим вариант с "Мужская", "Женская" и "Детская".

In [85]:
train.sex = train.sex.replace(['детский', 'Women'], ['Детский', 'Женский'])
train['sex'] = train.sex.replace(['Детский', 'Женский', 'Девочки', 'Мужской', 'Мальчики'], 
                                 ['для детей', 'для женщин', 'для девочек', 'для мужчин', 'для мальчиков'])
train.sex.value_counts()

для женщин       10614
для детей         4597
для мужчин        3672
для девочек       2379
для мальчиков     1226
Name: sex, dtype: int64

## Сезонность

In [75]:
def get_from_list(x: list[str]|str, 
                  params_seq: list[str]) -> float|int|str:
    """
    Обработка характеристик, где встречается несколько значения списком.
    """
    
    if isinstance(x, list):
        for param in params_seq:
            if param in x:
                return param
            return x[0]
    
    return x

In [76]:
train.season = train.season.apply(lambda x: get_from_list(x, ['лето', 'зима', 'демисезон', 'круглогодичный']))
train.season.value_counts()

круглогодичный    3193
лето              2574
демисезон         2318
зима              1829
Name: season, dtype: int64

## Хрупкость

In [80]:
def get_fragility(x):
    if x is None:
        return 'не указано'
    
    if re.match(r'не х[рупкое]+|нет|прочн|над[её]ж', x) is not None:
        return 'не хрупкий'
    elif re.match(r'х[рупкое]+|да|не брос', x) is not None:
        return 'хрупкий'
    else:
        return 'не хрупкий'

In [82]:
fragility = train[train.fragility.apply(type) != list].fragility.dropna()
fragility = fragility.apply(get_fragility)

fragility_lists = train[train.fragility.apply(type) == list].fragility
# обрабатываем списки, если встречается оба варианта - исключаем из рассмотрения
fragility_lists = fragility_lists.apply(lambda x: set([get_fragility(i) for i in x])).apply(lambda x: list(x)[0] if len(x) == 1 else None)

train['fragility'] = fragility

train['fragility'].value_counts()

не хрупкий    4773
хрупкий       2681
Name: fragility, dtype: int64

In [83]:
train['sex'] = train['sex'].astype('category')
train['age_restrictions'] = train['age_restrictions'].astype('category')
train['season'] = train['season'].astype('category')
train['fragility'] = train['fragility'].astype('category')

In [86]:
train[['sex', 'age_restrictions', 'season', 'fragility']].to_csv('chars_v1.csv', index=False)

In [19]:
train.to_parquet('train_preprocessed.parquet', index=False)
test.to_parquet('test_preprocessed.parquet', index=False)

# Схема валидации

Обучение на первом этапе происходит с использованием `contrastive learning`: косинусная схожесть векторных представлений текста и изображения для одного товара увеличивается, а для пар от разных товаров - уменьшается.

## Кластеризация
1. Снижение размерности векторов получаемых эмбеддингов изображений и текста: `PCA`, `UMAP` или `t-SNE`;
2. Кластеризация: `KMeans`, `HDBSCAN`;
3. Улучшение метрики *Силуэта*;
4. Проверка, относит ли к одному и тому же кластеру вектора картинок товаров, для которых заголовок и описание (или по-отдельности) полностью совпадают;
5. Расчет метрик `AMI` и V-меры для известных категорий товаров.

## Классификация
1. Данные: извлеченные из характеристик в train датасете возрастные ограничения, сезонность, хрупкость, пол. Разеляются на train и valid части.
2. Построение классификаторов только на основе текста или только на основе изображения. Получение метрик для унимодальных классификаторов. В зависимости от числа классов используется либо `wheighted-averaged-F1` либо `F1` метрика.;
3. На основе двух энкодеров строится `fusion` модель для классификации. Обучается на train части, тестируется на valid части. Результат сравнивается с унимодальными моделями.

# Литература:
- [Ye Bi et al. (2017) A Multimodal Late Fusion Model for E-Commerce Product Classification](https://arxiv.org/pdf/2008.06179.pdf)
- [Pasawee Wirojwatanakul et al. (2019) Multi-Label Product Categorization Using Multi-Modal Fusion Models](https://arxiv.org/abs/1907.00420)
- [Ekansh Verma (2020) Deep Multi-level Boosted Fusion Learning Framework for Multi-modal Product Classification](https://sigir-ecom.github.io/ecom20DCPapers/SIGIR_eCom20_DC_paper_8.pdf)
- [Wonyoung Shin (2022) e-CLIP: Large-Scale Vision-Language Representation Learning in E-commerce](https://arxiv.org/abs/2207.00208)
- [Нейронная Сеть CLIP от OpenAI: Классификатор, который не нужно обучать. Да здравствует Обучение без Обучения | Хабр](https://habr.com/ru/articles/539312/)
- [ruCLIP — мультимодальная модель для русского языка | Хабр](https://habr.com/ru/companies/sberdevices/articles/564440/)